`zepto_v2.csv`

In [54]:
import pandas as pd, numpy as np, plotly.express as px

In [55]:
df = pd.read_csv('zepto_v2.csv', encoding='cp1252')
colors = ["#4E79A7", "#F28E2B", "#E15759", "#76B7B2", "#59A14F", "#EDC949", "#AF7AA1", "#FF9DA7", "#9C755F", "#BAB0AC", "#1F77B4", "#FF7F0E", "#2CA02C", "#D62728", "#9467BD"]

In [56]:
print(df.max())
print(df.idxmax())
print(df.min())
print(df.idxmin())


Category                            Personal Care
name                      iD Wheat Lachha Paratha
mrp                                        260000
discountPercent                                51
availableQuantity                               6
discountedSellingPrice                     139900
weightInGms                                 10000
outOfStock                                   True
quantity                                     1500
dtype: object
Category                  2753
name                      1382
mrp                        517
discountPercent           2608
availableQuantity           53
discountedSellingPrice     517
weightInGms                108
outOfStock                  87
quantity                  2817
dtype: int64
Category                                                          Beverages
name                      "Godrej Aer Power Pocket - Long Lasting Bathro...
mrp                                                                       0
discountPercent 

In [57]:
dfN = df.drop_duplicates(subset='name').copy()
dfN.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1681 entries, 0 to 3728
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Category                1681 non-null   object
 1   name                    1681 non-null   object
 2   mrp                     1681 non-null   int64 
 3   discountPercent         1681 non-null   int64 
 4   availableQuantity       1681 non-null   int64 
 5   discountedSellingPrice  1681 non-null   int64 
 6   weightInGms             1681 non-null   int64 
 7   outOfStock              1681 non-null   bool  
 8   quantity                1681 non-null   int64 
dtypes: bool(1), int64(6), object(2)
memory usage: 119.8+ KB


In [58]:
labels = ['Category', 'discountPercent', 'availableQuantity']

In [59]:
for value in labels:
    print(f'Unique in {value}: ', df[f'{value}'].sort_values(ascending=True).unique(), 'with length of : ', len(df[f'{value}'].unique()))
    print()

Unique in Category:  ['Beverages' 'Biscuits' 'Chocolates & Candies' 'Cooking Essentials'
 'Dairy, Bread & Batter' 'Fruits & Vegetables' 'Health & Hygiene'
 'Home & Cleaning' 'Ice Cream & Desserts' 'Meats, Fish & Eggs' 'Munchies'
 'Paan Corner' 'Packaged Food' 'Personal Care'] with length of :  14

Unique in discountPercent:  [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 35 40 43 45 46 49 50 51] with length of :  42

Unique in availableQuantity:  [0 1 2 3 4 5 6] with length of :  7



In [60]:
dfN

,Category,name,mrp,discountPercent,availableQuantity,discountedSellingPrice,weightInGms,outOfStock,quantity
0,Fruits & Vegetables,Onion,2500,16,3,2100,1000,False,1
1,Fruits & Vegetables,Tomato Hybrid,4200,16,3,3500,1000,False,1
2,Fruits & Vegetables,Tender Coconut,5100,15,3,4300,58,False,1
3,Fruits & Vegetables,Coriander Leaves,2000,15,3,1700,100,False,100
4,Fruits & Vegetables,Ladies Finger,1400,14,3,1200,250,False,250
...,...,...,...,...,...,...,...,...,...
3717,Health & Hygiene,Kapiva Wild Tulsi Giloy Juice |,40000,15,1,34000,1000,False,1
3718,Health & Hygiene,Dabur Amla Juice,23500,15,6,19900,1000,False,1
3719,Health & Hygiene,Epigamia Fruit Yogurt Strawberry,4000,50,0,2000,75,True,75
3722,Health & Hygiene,"Eno Cooling Sachets - Cool Mint, 6 Pieces Carton",5400,11,0,4800,5,True,5


In [128]:
fig = px.histogram(dfN, x='Category', color = 'Category', color_discrete_sequence=colors, template='plotly_dark',height=750)
fig.show()

In [75]:
fig = px.histogram(dfN, x='Category', barmode='group', color = 'outOfStock', color_discrete_sequence=['green','red'], template='plotly_dark', height=750)
fig.show()


In [87]:
from plotly.subplots import make_subplots as mks
import plotly.graph_objs as go

In [111]:
dfN1 = dfN[dfN['Category']=='Cooking Essentials'].copy()
boolArray = ['True', 'False']
print(dfN1['outOfStock'].value_counts().tolist())


[420, 58]


In [158]:
uniqueCategories = dfN['Category'].unique()
n, m = 1, 1
colorPie = ['#ff6361','#ffa600']
subFig1 = mks(rows=1, cols=5, specs=[[{"type": "pie"}, {"type": "pie"}, {"type": "pie"}, {"type": "pie"}, {"type": "pie"}]])
subFig2 = mks(rows=1, cols=4, specs=[[{"type": "pie"}, {"type": "pie"}, {"type": "pie"}, {"type": "pie"}]])

# for value in uniqueCategories:
for value in uniqueCategories:
    if n<=5:
        dfTemp = dfN[dfN['Category'] == f'{value}']
        # figPie = px.pie(dfTemp, names='outOfStock', color ='outOfStock', color_discrete_sequence=colors[n:n+2], template='plotly_dark', width=500, title=f'{value}')
        countBool = dfTemp['outOfStock'].value_counts().tolist()
        subFig1.add_trace(
            go.Pie(values=countBool, labels=['True', 'False'], title=f'{value}'),
            row=1, col=n).update_traces(hoverinfo='label+value', textinfo='percent', marker=dict(colors=colorPie))
        n+=1
        print(f'The Probability of being out of stock for {value} is : ', round(((countBool[1]/countBool[0])*100), 5), '%')
    else:
        dfTemp = dfN[dfN['Category'] == f'{value}']
        countBool = dfTemp['outOfStock'].value_counts().tolist()
        print(f'The Probability of being out of stock for {value} is : ', round(((countBool[1]/countBool[0])*100), 5), '%')
        subFig2.add_trace(
            go.Pie(values=countBool, labels=['True', 'False'], title=f'{value}'),
            row=1, col=m).update_traces(hoverinfo='label+value', textinfo='percent', marker=dict(colors=colorPie))
        m+=1
subFig1.show()
subFig2.show()

# f.show()

The Probability of being out of stock for Fruits & Vegetables is :  4.70588 %
The Probability of being out of stock for Cooking Essentials is :  13.80952 %
The Probability of being out of stock for Dairy, Bread & Batter is :  30.12048 %
The Probability of being out of stock for Packaged Food is :  9.96169 %
The Probability of being out of stock for Meats, Fish & Eggs is :  28.0 %
The Probability of being out of stock for Biscuits is :  35.71429 %
The Probability of being out of stock for Personal Care is :  6.16438 %
The Probability of being out of stock for Home & Cleaning is :  9.46746 %
The Probability of being out of stock for Health & Hygiene is :  5.35714 %


In [153]:
# fig = px.pie(dfN, names='outOfStock', color ='outOfStock', color_discrete_sequence=colors, template='plotly_dark')
# countBool = dfN['outOfStock'].value_counts().tolist()
# print(countBool)
# print(f'The Probability of being out of stock for overall is : ', round(((countBool[1]/countBool[0])*100), 5), '%')
# fig.show()

In [160]:
fig = px.pie(dfN, names='availableQuantity', color = 'availableQuantity', color_discrete_sequence=colors, template='plotly_dark').update_traces(hoverinfo='label+value', textinfo='percent', marker=dict(colors=colorPie))
fig.show()

In [ ]:
fig = px.pie(dfN,names='discountPercent', color = 'discountPercent', color_discrete_sequence=colors, template='plotly_dark').update_traces(textposition='inside').update_layout(uniformtext_minsize=12, uniformtext_mode='hide')
fig.show()

In [205]:
dfCookingEssentials = df[df['Category'] == 'Cooking Essentials']
subFig = mks(rows=1, cols=2)
dfTemp = dfCookingEssentials.sort_values(by='quantity', ascending = False)
fig1 = go.Line(y = dfTemp['mrp'],line=dict(color='#bc5090', width=4),opacity=0.7)
fig2 = go.Line(y = dfTemp['discountedSellingPrice'],line=dict(color='#ffa600', width=4), opacity=0.7)
subFig.add_trace(fig1)
subFig.add_trace(fig2)
subFig.update_layout(autosize=True, showlegend=False).show()
dfTemp



C:\Users\Tanishq\PycharmProjects\zepto-inv-dataset\.venv\Lib\site-packages\plotly\graph_objs\_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




,Category,name,mrp,discountPercent,availableQuantity,discountedSellingPrice,weightInGms,outOfStock,quantity
482,Cooking Essentials,Jivana Jaggery,10000,0,2,10000,950,False,950
477,Cooking Essentials,Praakritik Organic Jaggery,16000,6,6,15000,900,False,900
408,Cooking Essentials,Saffola 100% Pure Honey,47500,8,2,43700,1000,False,750
233,Cooking Essentials,Hershey's Chocolate Flavour Syrup,22000,6,6,20500,623,False,680
226,Cooking Essentials,Hershey's Caramel Syrup,23000,0,1,23000,623,False,623
...,...,...,...,...,...,...,...,...,...
113,Cooking Essentials,Aashirvaad Multigrain Atta,6400,3,2,6200,1000,False,1
117,Cooking Essentials,Dhara Groundnut Oil (Pouch),23000,11,6,20300,1000,False,1
94,Cooking Essentials,iD Idli & Dosa Batter,8000,15,6,6800,1000,False,1
588,Cooking Essentials,Daawat Rozana Gold Basmati Rice,9000,1,0,8900,1000,True,1
